(case_studies)=
# Case studies

Description of the test cases
* *Motivate why they were selected as the candidates*
* *Where the data come from, measurement accuracy/uncertainty*
* *Describe the differences between the test cases to show how generally applicable the methodology is*
* *Success criteria: make a statement about which deviations or accuracy in the prediction are accepted as ''successful'' and why*


In [ ]:
# %load imports.py
%load_ext autoreload
%autoreload 2
%reload_kedro
%config Completer.use_jedi = False  ## (To fix autocomplete)
%matplotlib inline

import pandas as pd
pd.set_option('display.max_columns', 500)
from src.models.vmm import ModelSimulator
import matplotlib.pyplot as plt
from src.visualization.plot import track_plots, plot, captive_plot
import kedro
import numpy as np
import os.path
import anyconfig

import matplotlib
plt.style.use('paper')

from myst_nb import glue
from src.symbols import *
import src.symbols as symbols
from src.system_equations import *

from IPython.display import display, Math, Latex, Markdown
from sympy.physics.vector.printing import vpprint, vlatex

from src.parameters import df_parameters
p = df_parameters["symbol"]

# Read configs:
conf_path = os.path.join("../../conf/base/")
runs_globals_path = os.path.join(
    conf_path,
    "runs_globals.yml",
)

runs_globals = anyconfig.load(runs_globals_path)
model_test_ids = runs_globals["model_test_ids"]

join_globals_path = os.path.join(
    conf_path,
    "join_globals.yml",
)

joins = runs_globals["joins"]
join_runs_dict = anyconfig.load(join_globals_path)

globals_path = os.path.join(
    conf_path,
    "globals.yml",
)
global_variables = anyconfig.load(globals_path)



vmms = global_variables["vmms"]
from src.visualization.plot import track_plots, track_plot, plot, plot_parameters

In [ ]:
model_test_ids.keys()

In [ ]:
ships = ['wpcc','LNG','kvlcc2_hsva']

ship_datas = {}
initial_parameters = {}

for ship in ships:
       
    ship_datas[ship] = catalog.load(f"{ship}.ship_data")
    initial_parameters[ship] = catalog.load(f"{ship}.initial_parameters")
    

ship_datas = pd.DataFrame(ship_datas)
ship_datas.dropna(how='any', inplace=True)

initial_parameters = pd.DataFrame(initial_parameters)
initial_parameters.drop(index='Xthrust', inplace=True)
mask = (initial_parameters!=0).all(axis=1)
initial_parameters = initial_parameters.loc[mask].copy()

In [ ]:
ship_data_table = ship_datas.copy()
ship_data_table.rename(columns={'wpcc':'WPCC', 'kvlcc2_hsva': 'KVLCC2 (HSVA)'}, inplace=True)
ship_data_table = ship_data_table.transpose()
ship_data_table.drop(columns=['rho'], inplace=True)
ship_data_table.rename(
    columns={
        'T':'T [m]',
        'L':'L [m]',
        'B':'B [m]',
        'x_G': 'LCG [m]',
        'm': 'm [kg]',
        'I_z':'Izz [kg m2]',
        'volume':'Disp [m3]',
        'scale_factor':'scale factor',
        'TWIN':'Np',
    },
    
    inplace=True)

ship_data_table['Np']+=1
ship_data_table['Np']=ship_data_table['Np'].astype(int)

ship_data_table = ship_data_table.round(decimals=2)
glue("ship_datas", ship_data_table)

### Initial conditions and assumptions

In [ ]:
initial_parameter_table = initial_parameters.rename(columns={'wpcc':'WPCC', 'kvlcc2_hsva': 'KVLCC2 (HSVA)'})
initial_parameter_table = initial_parameter_table.transpose()
initial_parameter_table = initial_parameter_table.round(decimals=6)

glue("initial_parameters", 1000*initial_parameter_table)

In [ ]:
df_brix = pd.merge(left=initial_parameters, right=df_parameters, how='left', left_index=True, right_index=True)
df_brix.dropna(subset=['brix'], inplace=True)
brix = "brix_manoeuvring_1993"
df_brix['ref'] = brix

In [ ]:
s_table = """
| Parameter                            |       Equation                    | Reference |
|:------------------------------------:|:---------------------------------:|:---------------------------------:|
"""
for name, parameter in df_brix.iterrows():
    
    glue(name, parameter['brix'])
    s_table+= f"| ${parameter['symbol']}$                            |       {{glue:}}`{name}`                     | {{cite:p}}`{parameter['ref']}` |\n"
    
print(s_table)

Parameters in Abkowitz VMM:s are identified with the developed PIT for three test cases: WPCC, LNG and KVLCC2. Model test data from SSPA Maritime Dynamics Laboratory is used for WPCC, and LNG. Data from the SIMMAN2008 conference {cite:p}`stern_experience_2011` tested at HSVA is used for the KVLCC2. Main dimensions for the ship models used in these tests are summarized in [table](ship_datas).

```{glue:figure} ship_datas
:name: "ship_datas"

Ship data in model scale for WPCC, LNG tanker and KVLCC2.
```

### Initial guessed parameters

Following the PIT algorithm ([fig](overview)) first step is to make an initial guess for a linear VMM to be used in the the first EKF iteration.
The following semi-empirical formulas are used as initial guesses for hydrodynamic derivatives. 

| Parameter                            |       Equation                    | Reference |
|:------------------------------------:|:---------------------------------:|:---------------------------------:|
| $N_{r}$                            |       {glue:}`Nr`                     | {cite:p}`brix_manoeuvring_1993` |
| $N_{\dot{r}}$                            |       {glue:}`Nrdot`                     | {cite:p}`brix_manoeuvring_1993` |
| $N_{v}$                            |       {glue:}`Nv`                     | {cite:p}`brix_manoeuvring_1993` |
| $N_{\dot{v}}$                            |       {glue:}`Nvdot`                     | {cite:p}`brix_manoeuvring_1993` |
| $X_{\dot{u}}$                            |       {glue:}`Xudot`                     | {cite:p}`brix_manoeuvring_1993` |
| $Y_{r}$                            |       {glue:}`Yr`                     | {cite:p}`brix_manoeuvring_1993` |
| $Y_{\dot{r}}$                            |       {glue:}`Yrdot`                     | {cite:p}`brix_manoeuvring_1993` |
| $Y_{v}$                            |       {glue:}`Yv`                     | {cite:p}`brix_manoeuvring_1993` |
| $Y_{\dot{v}}$                            |       {glue:}`Yvdot`                     | {cite:p}`brix_manoeuvring_1993` |

Calculated values for the three test cases can be seen in [fig](initial_parameters)

```{glue:figure} initial_parameters
:name: "initial_parameters"

Initial guess for linear hydrodynamic derivatives for WPCC, LNG tanker and KVLCC2. (Values have been multiplied by 1000)
```

Hydrodynamic deriviatives in the Abkowitz model are identified with linear regression on forces and moments from the model tests inverse dynamics. A comparison between the inverse dynamic forces and prediction with the regressed model is shown for one of the KVLCC2 model tests in [fig](force_prediction).

```{glue:figure} force_prediction
:name: "force_prediction"

Forces and moment from model test inverse dynamics and prediction with the regressed model.
```


### Motion regression

In [ ]:
from wPCC_pipeline.pipelines.motion_regression.nodes import predict_force

vmm_name = 'vmm_abkowitz'
id = 'HSVA_CPMC_KVLCC2_Z_20_10_S'
ship = 'kvlcc2_hsva'

data_ek_smooth = catalog.load(f"{ship}.initial.{id}.data_ek_smooth")
added_masses = catalog.load(f"{ship}.added_masses")
ship_parameters = catalog.load(f"{ship}.ship_data")
vmm = catalog.load(vmm_name)
regression = catalog.load(f"{ship}.initial.{vmm_name}.joined.regression")
df_force = predict_force(data=data_ek_smooth, added_masses=added_masses, ship_parameters=ship_parameters, vmm=vmm) 

In [ ]:
data_prime = regression.ps.prime(df_force, U = df_force['U'])

X, _ = regression.diff_eq_X.calculate_features_and_label(
            data=data_prime, y=data_prime['u']
        )

X = X[regression.X_X.columns].copy()

data_prime['fx'] = regression.model_X.predict(X) + data_prime['thrust']*regression.exclude_parameters['Xthrust']

X = regression.diff_eq_Y.calculate_features(data_prime)
data_prime['fy'] = regression.model_Y.predict(X)

X = regression.diff_eq_N.calculate_features(data_prime)
data_prime['mz'] = regression.model_N.predict(X)


data_prediction = regression.ps.unprime(data_prime, U=df_force['U'])

fig,axes=plt.subplots(nrows=3)

units = {'fx':'N',
         'fy':'N',
         'mz':'Nm'}
for ax,dof in zip(axes,['fx','fy','mz']):

    df_force.plot(y=dof, label='inverse dynamics', ax=ax)
    data_prediction.plot(y=dof, label='prediction', ax=ax)
    
    label=f"${dof[0]}_{dof[1]}$ $[{units[dof]}]$"
    
    ax.set_ylabel(label)
    ax.get_legend().set_visible(False)
    
axes[0].legend(loc='lower right')
axes[-1].set_xlabel('time [s]')

axes[0].get_xaxis().set_visible(False)
axes[1].get_xaxis().set_visible(False)
plt.tight_layout()

glue("force_prediction",fig, display=False)

* *If needed: detailed description of each test case and ship model*

### Recursive Extended Kalman Filter

Simulation results with the initial guessed model and two iterrations of the PIT algorithm is shown in [fig](iterations). 
It can be seen that the first iteration of the PIT algorithm is much closer to the model test data compared to the initial guessed linear VMM model and the second iteration is even closer. The motion regression in the PIT algorithm has been run on all of the KVLCC2 model tests. 

```{glue:figure} iterations
:figwidth: 1000px
:name: "iterations"

Model test data for one of the KVLCC2 tests compared to simulations with initial model, PIT first iteration model and PIT second iteration model. 

```

In [ ]:
#id = '22773'
#ship = 'wpcc'
id = 'HSVA_CPMC_KVLCC2_Z_20_10_S'
ship = 'kvlcc2_hsva'

data_raw = catalog.load(f"{ship}.{id}.data")
data_lowpass = catalog.load(f"{ship}.{id}.data_lowpass")

data = catalog.load(f"{ship}.initial.{id}.data_ek_smooth")
ek = catalog.load(f"{ship}.vmm_linear.ek")
model = catalog.load(f"{ship}.initial.vmm_linear.joined.model")


In [ ]:
data['U'] = np.sqrt(data['u']**2 + data['v']**2)
df_sim = ek.simulate(data=data, input_columns=['delta','U'], solver='Radau')

dataframes = {
    'initial guessed model': df_sim,
    'PIT iteration 1': catalog.load(f"{ship}.initial.{vmm_name}.joined.{id}.data_resimulate"),
    'PIT iteration 2': catalog.load(f"{ship}.updated.{vmm_name}.joined.{id}.data_resimulate"),
    'model test' : data,
}

keys = ['u','v','r']
error_keys = [f'$\\epsilon({key})$' for key in keys]  
for key, df_ in dataframes.items():
    df_[error_keys]=df_[keys] - data[keys]

fig,ax=plt.subplots()
track_plots(dataframes, lpp=ship_datas[ship]['L'], beam=ship_datas[ship]['B'], time_window=[0,70], N=2, ax=ax);
glue("iterations",fig, display=False)

fig = plot(dataframes, keys=error_keys, ncols=1);
plt.tight_layout()
glue("iteration_errors",fig, display=False)

rmse = {}
rmse_keys = [f'$RMSe({key})$' for key in keys] 
for key, df_ in dataframes.items():
    rmse[key] = np.sqrt((df_[error_keys]**2).mean())
    

rmse = pd.DataFrame(rmse)

r = rmse.loc['$\\epsilon(r)$']
u = rmse.loc['$\\epsilon(u)$']
v = rmse.loc['$\\epsilon(v)$']
rmse.loc['total'] = u+v+r*ship_datas[ship]['L']/2